In [9]:
import re
import nltk
import pandas as pd
import numpy as np
#nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ryanmurray/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import pickle

FN0 = 'vocabulary-embedding'

with open('data/%s.pkl'%FN0, 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)
    
vocab_size, embedding_size = embedding.shape

with open('data/%s.data.pkl'%FN0, 'rb') as fp:
    X,Y = pickle.load(fp)

In [3]:
def filter_with_glove(X = X,min_idx=200):
    transcripts = []
    for t in X:
        talk = []
        for w in t:
            try:
                g = glove_idx2idx[w]
            except: g = w
            if g > min_idx:
                talk.append(idx2word[g])
        transcripts.append(' '.join(talk))
    return transcripts

In [4]:
transcripts = filter_with_glove(X,0)

In [6]:
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

In [14]:
#remove proper names
%time preprocess = [strip_proppers_POS(doc) for doc in transcripts]

CPU times: user 4min 51s, sys: 376 ms, total: 4min 52s
Wall time: 4min 52s


In [15]:
def tokenize_and_stem(text):
    #First tokenizes by sentence, then by word, so that punctuation is caught as its own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        #Filter out tokens not containing letters
        if re.search('[a-zA-Z]' ,token):
            filtered_tokens.append(token)
    return filtered_tokens

In [18]:
preprocess_text = [' '.join(text) for text in preprocess]

In [19]:

#tokenize
%time tokenized_text = [tokenize_and_stem(text) for text in preprocess_text]

CPU times: user 1min 3s, sys: 236 ms, total: 1min 3s
Wall time: 1min 3s


In [74]:
stopwords = nltk.corpus.stopwords.words('english')
upper_stop = [s.title() for s in stopwords]

total_stop = stopwords + upper_stop
#remove stop words
%time texts = [[word for word in text if word not in total_stop and len(word) > 3] for text in tokenized_text]

CPU times: user 21.9 s, sys: 0 ns, total: 21.9 s
Wall time: 21.9 s


In [85]:
with open('transcript_text.pkl','wb') as pf:
    pickle.dump(texts,pf)

In [28]:
from gensim import corpora, models, similarities 

In [75]:
dictionary = corpora.Dictionary(texts)

In [76]:
dictionary.filter_extremes(no_below=5, no_above=0.5)

In [77]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
%time lda = models.LdaMulticore(corpus, num_topics=25, id2word=dictionary, chunksize=10000, passes=50,workers=7)

In [60]:


topics_matrix = lda.show_topics(formatted=False, num_words=20)


'topics_matrix = np.array(topics_matrix)\n\ntopic_words = topics_matrix[:,:,1]\nfor i in topic_words:\n    print([str(word) for word in i])\n    print()\n'

In [62]:
import pyLDAvis
import pyLDAvis.gensim

In [91]:
with open('ldavis.pkl','wb') as picklefile:
    pickle.dump((lda,corpus,dictionary),picklefile)

In [89]:
vis = pyLDAvis.gensim.prepare(lda,corpus,dictionary,mds='mmds')

/home/ryanmurray/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [90]:
pyLDAvis.display(vis)

In [53]:
upper_stop = [s.title() for s in stopwords]

['I',
 'Me',
 'My',
 'Myself',
 'We',
 'Our',
 'Ours',
 'Ourselves',
 'You',
 "You'Re",
 "You'Ve",
 "You'Ll",
 "You'D",
 'Your',
 'Yours',
 'Yourself',
 'Yourselves',
 'He',
 'Him',
 'His',
 'Himself',
 'She',
 "She'S",
 'Her',
 'Hers',
 'Herself',
 'It',
 "It'S",
 'Its',
 'Itself',
 'They',
 'Them',
 'Their',
 'Theirs',
 'Themselves',
 'What',
 'Which',
 'Who',
 'Whom',
 'This',
 'That',
 "That'Ll",
 'These',
 'Those',
 'Am',
 'Is',
 'Are',
 'Was',
 'Were',
 'Be',
 'Been',
 'Being',
 'Have',
 'Has',
 'Had',
 'Having',
 'Do',
 'Does',
 'Did',
 'Doing',
 'A',
 'An',
 'The',
 'And',
 'But',
 'If',
 'Or',
 'Because',
 'As',
 'Until',
 'While',
 'Of',
 'At',
 'By',
 'For',
 'With',
 'About',
 'Against',
 'Between',
 'Into',
 'Through',
 'During',
 'Before',
 'After',
 'Above',
 'Below',
 'To',
 'From',
 'Up',
 'Down',
 'In',
 'Out',
 'On',
 'Off',
 'Over',
 'Under',
 'Again',
 'Further',
 'Then',
 'Once',
 'Here',
 'There',
 'When',
 'Where',
 'Why',
 'How',
 'All',
 'Any',
 'Both',
 'Each

In [83]:
with open('lda50.pkl', 'wb') as fp:
    pickle.dump(lda,fp)


In [84]:
with open('ldavis.pkl', 'wb') as fp:
    pickle.dump(vis,fp)

In [86]:
lda.perplexity(text)

AttributeError: 'LdaMulticore' object has no attribute 'perplexity'